## Create Example Jobs

Create various types of example jobs:

In [1]:
from scm.plams import from_smiles, AMSJob, PlamsError, Settings, Molecule, Atom
from scm.libbase import UnifiedChemicalSystem as ChemicalSystem
from scm.input_classes.drivers import AMS
from scm.input_classes.engines import DFTB
from scm.utils.conversions import plams_molecule_to_chemsys

def example_job_dftb(smiles, task, use_chemsys = False):
    # Generate molecule from smiles
    mol = from_smiles(smiles)
    if use_chemsys:
        mol = plams_molecule_to_chemsys(mol)

    # Set up calculation settings using PISA
    sett = Settings()
    sett.runscript.nproc = 1
    driver = AMS()
    driver.Task = task
    driver.Engine = DFTB() 
    sett.input = driver

    return AMSJob(molecule = mol, settings = sett)


def example_job_adf(smiles, task, basis, gga = None, use_chemsys = False):
    # Generate molecule from smiles
    mol = from_smiles(smiles)
    if use_chemsys:
        mol = plams_molecule_to_chemsys(mol)

    # Set up calculation settings using standard settings
    sett = Settings()
    sett.runscript.nproc = 1
    sett.input.AMS.Task = task
    sett.input.ADF.Basis.Type = basis
    if gga:
        sett.input.ADF.XC.GGA = gga
    return AMSJob(molecule = mol, settings = sett)
    

def example_job_neb(iterations, use_chemsys = False):
    # Set up molecules
    main_molecule = Molecule()
    main_molecule.add_atom(Atom(symbol="C", coords=(0, 0, 0)))
    main_molecule.add_atom(Atom(symbol="N", coords=(1.18, 0, 0)))
    main_molecule.add_atom(Atom(symbol="H", coords=(2.196, 0, 0)))
    final_molecule = main_molecule.copy()
    final_molecule.atoms[1].x = 1.163
    final_molecule.atoms[2].x = -1.078

    mol = {"": main_molecule, "final": final_molecule}

    if use_chemsys:
        mol = {k: plams_molecule_to_chemsys(v) for k, v in mol.items()}

    # Set up calculation settings
    sett = Settings()
    sett.runscript.nproc = 1
    sett.input.ams.Task = "NEB"
    sett.input.ams.NEB.Images = 9
    sett.input.ams.NEB.Iterations = iterations
    sett.input.DFTB
        
    return AMSJob(molecule = mol, settings = sett)

Run example jobs:

In [2]:
from scm.plams import config, JobRunner
config.default_jobrunner = JobRunner(parallel=True, maxthreads = 8)

smiles = ["CC", "C", "O", "CO"]
tasks = ["SinglePoint", "GeometryOptimization"]
engines = ["DFTB", "ADF"]
jobs = []
for i, s in enumerate(smiles):
    for j, t in enumerate(tasks):
        job_dftb = example_job_dftb(s, t, use_chemsys = i % 2)
        job_adf1 = example_job_adf(s, t, "DZ", use_chemsys = True)
        job_adf2 = example_job_adf(s, t, "TZP", "PBE")
        jobs += [job_dftb, job_adf1, job_adf2]

job_neb1 = example_job_neb(10)
job_neb2 = example_job_neb(100, use_chemsys=True)
jobs += [job_neb1, job_neb2]

for j in jobs:
    j.run()

[21.01|12:15:47] JOB plamsjob STARTED
[21.01|12:15:47] JOB plamsjob STARTED
[21.01|12:15:47] JOB plamsjob STARTED
[21.01|12:15:47] JOB plamsjob STARTED
[21.01|12:15:47] JOB plamsjob STARTED
[21.01|12:15:47] Renaming job plamsjob to plamsjob.002
[21.01|12:15:47] JOB plamsjob STARTED
[21.01|12:15:47] JOB plamsjob STARTED
[21.01|12:15:47] Renaming job plamsjob to plamsjob.003
[21.01|12:15:47] JOB plamsjob STARTED
[21.01|12:15:47] JOB plamsjob RUNNING
[21.01|12:15:47] JOB plamsjob STARTED
[21.01|12:15:47] Renaming job plamsjob to plamsjob.004
[21.01|12:15:47] Renaming job plamsjob to plamsjob.005
[21.01|12:15:47] Renaming job plamsjob to plamsjob.006
[21.01|12:15:47] JOB plamsjob.002 RUNNING
[21.01|12:15:47] Renaming job plamsjob to plamsjob.007
[21.01|12:15:47] Renaming job plamsjob to plamsjob.008
[21.01|12:15:47] JOB plamsjob.005 RUNNING
[21.01|12:15:47] JOB plamsjob.006 RUNNING
[21.01|12:15:47] JOB plamsjob.008 RUNNING
[21.01|12:15:47] JOB plamsjob.003 RUNNING
[21.01|12:15:47] JOB plam

In [3]:
for j in jobs:
    j.results.wait()

[21.01|12:16:01] Waiting for job plamsjob.004 to finish
[21.01|12:16:13] Waiting for job plamsjob.008 to finish


Delete the .dill file associated with some jobs:

In [14]:
from pathlib import Path
import os 

for j in jobs:
    if j.name.endswith(".005"):
        path = Path(j.path) / f"{j.name}.dill"
        os.remove(path)

## Job Analysis

### Loading Jobs

Jobs can be loaded by passing job objects, or loading from a path.

In [15]:
from scm.plams import JobAnalysis

In [20]:
ja = JobAnalysis(jobs = jobs[:4], paths = [j.path for j in jobs[4:]])

In [25]:
print(ja.to_table())

[21.01|12:29:52] Job plamsjob.025 reported errors. Please check the output
[21.01|12:29:52] Job plamsjob.025 reported errors. Please check the output
| Path                                                                                               | Name         | OK   | Check | ErrorMsg                          | Formula           | Smiles | CPUTime   | SysTime  | ElapsedTime         |
|----------------------------------------------------------------------------------------------------|--------------|------|-------|-----------------------------------|-------------------|--------|-----------|----------|---------------------|
| /Users/ormrodmorley/Documents/code/plams/scm/plams/examples/JobAnalysis/plams_workdir/plamsjob     | plamsjob     | True | True  | None                              | C2H6              | CC     | 0.225823  | 0.034566 | 0.27295899391174316 |
| /Users/ormrodmorley/Documents/code/plams/scm/plams/examples/JobAnalysis/plams_workdir/plamsjob.003 | plamsjob.003 | Tru